<a href="https://colab.research.google.com/github/cph316/generative_ai/blob/main/%E3%80%90HW9%E3%80%910415_AI%E4%BB%A3%E7%90%86%E8%A8%AD%E8%A8%88%E6%A8%A1%E5%BC%8F_%E5%A5%87%E6%83%B3%E8%A7%92%E8%89%B2%E8%A7%80%E9%BB%9E%E6%A9%9FTwo_Stage_CoT%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🎯 任務說明：將「奇想角色觀點機」改寫為 Chain-of-Thought（CoT）推理過程

讓我們換個腦袋來看事情！透過 AI 模型協作，這個工具將幫你用創意角色視角重新詮釋一段日常經歷。

#### ✅ CoT 改寫版本流程：
🧠 第一階段｜角色視角思考（Planner）
系統會接收你輸入的事件，並由「思考模型」發想出兩個虛擬角色（可以是人、動物、物品、幻想生物等），
每個角色都會用一句話，從自身角度來解讀這件事，可能是搞笑的、可愛的，或超展開的荒謬視角。

📌 目的：創造多角度理解，啟動換位思考與想像力

✍️ 第二階段｜角色 Po 文生成（Writer）
系統會從上述角色中選出一個最有趣的，並以該角色的第一人稱口吻，撰寫一段貼近社群風格的「Po 文」。
貼文會包含情緒表達（emoji）、角色語氣與主觀情感，並結尾加上適合的 hashtag。

📌 目的：讓角色具象化，體驗「如果我是它，會怎麼說這句話」的共感與創作


#### 1. 讀入金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
# 【使用 OpenAI】
api_key = userdata.get('OpenAI')
os.environ['OPENAI_API_KEY']=api_key
provider = "openai"
model = "gpt-4.1-nano"

#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.13.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 使用 AISuite 的準備

In [4]:
import aisuite as ai

In [5]:
provider_planner = "openai"
model_planner="gpt-4.1-nano"

provider_writer = "groq"
model_writer = "llama3-70b-8192"

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [7]:
system_planner = "用戶會輸入一件日常小事，請你用繁體中文發想兩個角色視角（人、動物、物品或幻想角色），並用一句話寫出他們對這件事的反應。語氣可以幽默、誇張、可愛或有創意。只要直接列出角色與反應句，例如：- 【流浪貓】：這電梯每次開門都沒罐罐，我失望。- 【AI攝影機】：我已經第128次紀錄你們尷尬對視了。"
system_writer = (
    "請用繁體中文，用某個角色的視角，寫一則第一人稱社群貼文，結合 emoji 與 hashtag，不超過 80 字。"
    "語氣請貼近該角色個性，可幽默、詩意或真誠。"
    "輸入格式為：角色：【角色名】，觀點：【事件詮釋】請直接生成貼文內容。"
)


In [8]:
def lucky_post(prompt):
    # Step 1: 使用 planner 模型產出兩個角色視角
    planning_prompt = f"使用者說：{prompt}"

    lucky_reasons_raw = reply(system_planner, planning_prompt,
                              provider=provider_planner,
                              model=model_planner)

    # 清理格式：將字串中的 "\\n" 轉換為真正換行
    lucky_reasons = lucky_reasons_raw.replace("\\n", "\n").strip()

    # Step 2: 把角色觀點直接丟給 writer 模型，請它選一個寫貼文
    generation_prompt = f"以下是兩個角色視角：\n{lucky_reasons}"

    lucky_post_raw = reply(system_writer, generation_prompt,
                           provider=provider_writer,
                           model=model_writer)

    lucky_post = lucky_post_raw.replace("\\n", "\n").strip()

    return lucky_reasons, lucky_post

In [9]:
lucky_post("今天下雨, 但居然忘帶傘了!!!")

('- 【流浪貓】：沒傘的我，現在就像一場落水的演出，唯一的幕新人是我自己！  \n- 【老舊雨鞋】：終於有人來慰問我這雙被忽視的伙伴，我還是值得一試的！',
 'Here are two social media posts from different character perspectives:\n\n**流浪貓**：雨雨雨！😭沒傘的我，現在就像一場落水的演出，唯一的幕新人是我自己！ 🤣 #落水貓 #雨天哀怨\n\n**老舊雨鞋**： finally！ 🙌 終於有人來慰問我這雙被忽視的伙伴，我還是值得一試的！ 💪 #雨鞋復仇 #老舊不廢')

### 4. 用 Gradio 打造對話機器人 Web App!

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00


In [11]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("### 🧠 角色視角創作機")
    gr.Markdown("請輸入一件小事，讓 AI 幫你從不同角色的觀點重新詮釋，看看世界的多樣與幽默！")

    user_input = gr.Textbox(label="請輸入一件日常小事")
    btn = gr.Button("生成角色視角與貼文 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🎭 兩個角色的詮釋觀點", lines=4)
        out2 = gr.Textbox(label="✍️ 角色貼文（第一人稱視角）", lines=3)

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [12]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://eb210a8f03469659e3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://eb210a8f03469659e3.gradio.live
